In [2]:
from diffusers import AutoencoderKL
import torch
from PIL import Image
import os
import numpy as np
import shutil

# Загрузка VAE
vae = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae", torch_dtype=torch.float16).to("cuda")

# Путь к данным
input_dir = "dataset_stub_4k"
output_latent_dir = "./datadir/latents"
output_prompt_dir = "./datadir/prompts"

os.makedirs(output_latent_dir, exist_ok=True)
os.makedirs(output_prompt_dir, exist_ok=True)

# Проходим по всем изображениям
for i in range(1, 4001):
    base_name = f"image_{i:05d}"
    png_path = os.path.join(input_dir, f"{base_name}.png")
    txt_path = os.path.join(input_dir, f"{base_name}.txt")

    # Читаем изображение
    image = Image.open(png_path).convert("RGB")
    # Нормализация: [0,255] → [-1, 1]
    image_tensor = torch.tensor(np.array(image)).permute(2, 0, 1).unsqueeze(0).float() / 127.5 - 1
    image_tensor = image_tensor.to("cuda", dtype=torch.float16)

    # Кодирование в латент
    with torch.no_grad():
        latents = vae.encode(image_tensor).latent_dist.sample()

    # Сохраняем латент
    torch.save(latents.cpu(), os.path.join(output_latent_dir, f"{base_name}.pt"))

    # Копируем промпт
    shutil.copy(txt_path, os.path.join(output_prompt_dir, f"{base_name}.txt"))

    if i % 100 == 0:
        print(f"✅ {i}/4000")

print("🎉 Все латенты и промпты сохранены!")


✅ 100/4000
✅ 200/4000
✅ 300/4000
✅ 400/4000
✅ 500/4000
✅ 600/4000
✅ 700/4000
✅ 800/4000
✅ 900/4000
✅ 1000/4000
✅ 1100/4000
✅ 1200/4000
✅ 1300/4000
✅ 1400/4000
✅ 1500/4000
✅ 1600/4000
✅ 1700/4000
✅ 1800/4000
✅ 1900/4000
✅ 2000/4000
✅ 2100/4000
✅ 2200/4000
✅ 2300/4000
✅ 2400/4000
✅ 2500/4000
✅ 2600/4000
✅ 2700/4000
✅ 2800/4000
✅ 2900/4000
✅ 3000/4000
✅ 3100/4000
✅ 3200/4000
✅ 3300/4000
✅ 3400/4000
✅ 3500/4000
✅ 3600/4000
✅ 3700/4000
✅ 3800/4000
✅ 3900/4000
✅ 4000/4000
🎉 Все латенты и промпты сохранены!


In [1]:
from transformers import CLIPTextModel, CLIPTokenizer
import torch

# Загрузка модели CLIP для эмбеддингов текста
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch16")
text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch16")

# Функция для преобразования текста в эмбеддинги
def get_text_embeddings(prompts):
    inputs = tokenizer(prompts, padding=True, return_tensors="pt", truncation=True)
    text_embeddings = text_model(**inputs).last_hidden_state.to("cuda")
    return text_embeddings


c:\newTry2\condaR\envs\sdxl-turbo-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os

# Датасет для латентов и текстовых подсказок
class LatentPromptDataset(Dataset):
    def __init__(self, latents_dir, prompts_dir):
        self.latents_dir = latents_dir
        self.prompts_dir = prompts_dir
        self.latent_files = sorted(os.listdir(latents_dir))  # Сортируем файлы для правильного порядка
        self.prompt_files = sorted(os.listdir(prompts_dir))  # Сортируем файлы для правильного порядка

    def __len__(self):
        return len(self.latent_files)

    def __getitem__(self, idx):
        # Путь к файлам
        latent_path = os.path.join(self.latents_dir, self.latent_files[idx])
        prompt_path = os.path.join(self.prompts_dir, self.prompt_files[idx])

        # Загружаем латенты и текстовый промпт
        latents = torch.load(latent_path)
        with open(prompt_path, 'r') as f:
            prompt = f.read().strip()

        return latents, prompt

# Папка с латентами и промптами
latents_dir = "./datadir/latents"  # Папка с латентами
prompts_dir = "./datadir/prompts"  # Папка с текстовыми подсказками

# Создаем DataLoader для батчевой обработки данных
dataset = LatentPromptDataset(latents_dir, prompts_dir)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


KeyboardInterrupt: 

In [2]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import torch
from transformers import CLIPTokenizer, CLIPTextModel
import torchvision.transforms as transforms

# Загрузка модели CLIP для эмбеддингов текста
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch16")
text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch16")

# Функция для преобразования текста в эмбеддинги
def get_text_embeddings(prompts):
    # Убедимся, что prompts - это список строк
    if isinstance(prompts, str):  # Если это одна строка
        prompts = [prompts]  # Преобразуем в список строк

    inputs = tokenizer(prompts, padding=True, return_tensors="pt", truncation=True)
    text_embeddings = text_model(**inputs).last_hidden_state.to("cuda")
    return text_embeddings

# Папка с изображениями и текстовыми файлами с промптами
images_dir = r"C:\newTry2\dataset_stub_4k"  # Путь к папке с изображениями
prompts_dir = r"C:\newTry2\dataset_stub_4k"  # Путь к папке с текстовыми файлами

# Преобразование изображений в 4D тензор
def image_to_tensor(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Подгонка размера под модель
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image_tensor = transform(image).unsqueeze(0).to("cuda")  # Добавляем batch dimension и отправляем на GPU
    return image_tensor

# Датасет для изображений и текстовых подсказок
class LatentPromptDataset(Dataset):
    def __init__(self, images_dir, prompts_dir):
        self.images_dir = images_dir
        self.prompts_dir = prompts_dir
        self.image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])
        self.prompt_files = sorted([f for f in os.listdir(prompts_dir) if f.endswith('.txt')])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Путь к файлам
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        prompt_path = os.path.join(self.prompts_dir, self.prompt_files[idx])

        # Загружаем изображение и текстовый промпт
        image_tensor = image_to_tensor(image_path)  # Преобразуем изображение в 4D тензор
        with open(prompt_path, 'r') as f:
            prompt = f.read().strip()  # Читаем строку промпта

        return image_tensor, prompt

# Создаем DataLoader для батчевой обработки данных
dataset = LatentPromptDataset(images_dir, prompts_dir)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


c:\newTry2\condaR\envs\sdxl-turbo-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
import torch
from micro_diffusion.micro_diffusion.models.model import create_latent_diffusion

# Проверим память до загрузки модели
print(f"GPU память до: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

# Загружаем модель учителя на CPU
teacher_model = create_latent_diffusion(
    latent_res=64,
    in_channels=4,
    pos_interp_scale=2.0,
    precomputed_latents=True,
    # text_latents_key="caption_latents", 
    dtype="float32"  # Указание типа данных для совместимости
).to("cpu")

# Загружаем веса модели учителя с игнорированием несовпадений
teacher_model.dit.load_state_dict(
    torch.load("./micro_diffusion/trained_models/teacher.pt", map_location="cpu"),
    strict=False  # Игнорируем несовпадения в ключах
)

# Переводим модель в режим оценки (inference)
teacher_model.eval()

# Проверим память после загрузки модели
print("✅ Учитель загружен на CPU")
print(f"GPU память после: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")


GPU память до: 8.47 GB


c:\newTry2\condaR\envs\sdxl-turbo-env\lib\site-packages\open_clip\factory.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_

✅ Учитель загружен на CPU
GPU память после: 8.43 GB


In [21]:
def generate_teacher_latents(image, text_embeddings):
    with torch.no_grad():
        # Формируем правильный словарь для модели
        batch = {
            "image": image,  # Передаем изображения (не латенты)
            "caption_latents": text_embeddings  # Эмбеддинги текста
        }
        
        # Подаем данные в модель учителя
        teacher_latents = teacher_model(batch)  # Передаем как словарь
    return teacher_latents


In [5]:
# Преобразование изображения в 4D тензор
def image_to_tensor(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Подгонка размера под модель
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image_tensor = transform(image).unsqueeze(0).to("cuda")  # Добавляем batch dimension и отправляем на GPU
    return image_tensor


In [6]:
# Импортируем необходимые библиотеки
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt  # Для графика
from torch.utils.data import DataLoader

# Настройка модели студента на GPU
student_model = create_latent_diffusion(
    latent_res=64,
    in_channels=4,
    pos_interp_scale=2.0,
    dtype="float32"  # Использование float32 для точности
).to("cuda")

# Определение оптимизатора для модели студента
optimizer = optim.Adam(student_model.parameters(), lr=1e-5)

# Функция потерь для консистентности между учителем и студентом
def consistency_loss(student_latents, teacher_latents):
    return nn.MSELoss()(student_latents, teacher_latents)


In [12]:
# Лист для хранения значений лосса
losses = []

# Функция для тренировки модели студента
def train_student_model(dataloader, teacher_model, num_epochs):
    student_model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0  # Сумма потерь за эпоху
        for i, (images, prompts) in enumerate(dataloader):  # Теперь images, а не latents
            images = images.to("cuda")[:,0,...]
            text_embeddings = get_text_embeddings(prompts).to("cuda")
            print(images.shape, text_embeddings.shape)
            # Генерация латентов моделью учителя (на CPU)
            teacher_latents = generate_teacher_latents(images.to("cpu"), text_embeddings.to("cpu"))

            # Генерация латентов моделью студента (на GPU)
            student_latents = student_model(images, text_embeddings)

            # Вычисление потерь
            loss = consistency_loss(student_latents, teacher_latents)
            epoch_loss += loss.item()

            # Обновление весов модели студента
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 100 == 0:
                print(f"Эпоха {epoch+1}, Итерация {i}, Потери: {loss.item()}")

        # Сохраняем потери за эпоху
        losses.append(epoch_loss / len(dataloader))  # Среднее значение потерь за эпоху
        print(f"Эпоха {epoch+1}, Средний лосс: {epoch_loss / len(dataloader)}")

    print("Обучение завершено!")


In [22]:
# Пример тренировки
num_epochs = 10  # Установите количество эпох
train_student_model(dataloader, teacher_model, num_epochs)


torch.Size([16, 3, 256, 256]) torch.Size([16, 5, 512])


AssertionError: Input height (32) doesn't match model (64).

In [ ]:
# График лосса
plt.plot(range(1, num_epochs+1), losses)
plt.xlabel("Эпохи")
plt.ylabel("Средний лосс")
plt.title("График изменения лосса")
plt.show()